In [91]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

%matplotlib inline

In [92]:
def region_of_interest(image, vertices):
    mask = np.zeros_like(image)
    channels_count = image.shape[-1] if len(image.shape) == 3 else 1
    match_mask_color = (255,) * channels_count

    cv.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv.bitwise_and(image, mask)

    return masked_image

In [93]:
def draw_the_lines(img_src, hough_lines_p):
    image = np.copy(img_src)
    blank_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    for line in hough_lines_p:
        for x1, y1, x2, y2 in line:
            cv.line(blank_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    img = cv.addWeighted(image, .8, blank_image, 1, 0.0)

    return img

In [94]:
def process(img):
    height = img.shape[0]
    width = img.shape[1]

    region_of_interest_vertices = [
        (100, height),
        (width/2 - width*0.04, height/2 + height*0.01),
        (width-100, height)
    ]

    img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    edges = cv.Canny(img_gray, 120, 255, apertureSize=3)
    cropped_image = region_of_interest(edges, np.array([region_of_interest_vertices], np.int32))
    lines = cv.HoughLinesP(cropped_image, 1, np.pi/180, 40, minLineLength=5, maxLineGap=20)
    image_with_lines = draw_the_lines(img, lines)

    return image_with_lines

In [95]:
cap = cv.VideoCapture('video/road.mp4')
cv.namedWindow('frame', cv.WINDOW_NORMAL)

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        frame = process(frame)
        cv.imshow('frame', frame)

    else:
        print('no video')
        cap.set(cv.CAP_PROP_POS_FRAMES, 0)
        continue

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

no video
no video
no video
no video


In [96]:
cap.release()
cv.destroyAllWindows()